# AEM-as-a-CloudService - CDN Cache Hit Ratio Analysis - Jupyter Notebook

A quickstart sample Jupyter Notebook to analyze your AEMCS CDN logs. The analysis shows the **Cache HIT ratio** for the following request types and provides a list of the **top URLs of MISS and PASS cache types**.

- Total 
- HTML 
- Image
- JavaScript
- CSS
- JSON

>
> **IMPORTANT::**
> Please change the value of `json_file` variable in the next block to analyze your CDN log file.
>

The CDN log file is downloaded from the Adobe Cloud Manager. 

Feel free to edit this Jupyter Notebook file to gain further insight and optimize the CDN configurations.

## Install additional libraries 

In [ ]:
!pip install termcolor
!pip install tabulate

## Load CDN Log File

In [ ]:
import pandas as pd
import numpy as np
import csv
import json
import re
import matplotlib.pyplot as plt

from termcolor import colored


# ********************************************************
# IMPORTANT - Update the below `log_file` value with your log file name/path.
# ********************************************************
log_file = 'publish_cdn_2023-09-22.log'

In [ ]:
# Setting Pandas options
pd.set_option('display.max_columns', 100) 
pd.set_option('display.max_colwidth', 100) 

data = []
col_names = ['timestamp', 'ttfb', 'cli_ip', 'cli_country', 'rid', 'req_ua', 'host', 'url', 'method', 'res_ctype', 'cache', 'res_age', 'status', 'pop', 'rules']

# Patterns to ignore log lines
log_line_patterns_to_ignore = [r'(?:\"url\":"\/systemready\")', r'(?:\"cache\":\"SYNTH\")']

# Read the log file
with open(log_file, encoding='utf-8') as in_file:
    for line in in_file:
        if not any(re.search(pattern, line) for pattern in log_line_patterns_to_ignore):
            log = json.loads(line)
            data.append(log)

# Create DataFrame
df = pd.DataFrame(data=data, columns=col_names)

# Display the first three rows, to review the CDN log details
df.head(3)

## Perform Analysis

### Display Analysis Result for Total, HTML, JS/CSS and Image Requests

For each request type (HTML, JS/CSS, Image) the analysis result contains
- Cache Hit Ratio
- Pie chart
- Bar chart

In [ ]:
# Graph related definitions
labels = ['Hit', 'Pass', 'Miss']
colors = ['green', 'blue','red']  
bar_colors = ['tab:green', 'tab:blue','tab:red']

def analyze(total_count, miss_count, hit_count, pass_count, request_type):

    # Calculate and print the cacheRatio
    cache_ratio = (hit_count*100/(total_count))
    
    print("\n\n")
    print(colored("=================================================================================", "red", attrs=["bold"]))
    print(colored(f"The {request_type} cache hit ratio is: {cache_ratio:.2f}%", "red", attrs=["bold"]))
    print(colored("=================================================================================", "red", attrs=["bold"]))

    #print(colored(f"\n \n \t {request_type} cache hit ratio is: {cache_ratio:.2f}% \n", "red", attrs=["bold","underline"]))

    displayPieChart(hit_count, pass_count, miss_count, total_count, request_type)

    displayBarGraph(hit_count, pass_count, miss_count, request_type)
    

def displayPieChart(hit_count, pass_count, miss_count, total_count, request_type):
    
    # Display the cache hit ratio as a pie chart
    hit_percent = (hit_count*100/(total_count))
    pass_percent = (pass_count*100/(total_count))
    miss_percent = (miss_count*100/(total_count))

    # Data for the pie chart
    sizes = [hit_percent, pass_percent, miss_percent]  

    plt.pie(sizes, labels=labels, colors=colors, autopct='%1.1f%%', startangle=140)

    # Add labels and title
    plt.title(f'{request_type} - CDN Cache Percentage')

    # Show the plot
    plt.show()

def displayBarGraph(hit_count, pass_count, miss_count, request_type):
    # Display the bar graph
    fig, ax = plt.subplots()
    counts = [hit_count, pass_count, miss_count]

    # Create the plot
    ax.bar(labels, counts, label=labels, color=bar_colors)

    ax.set_ylabel('Count')
    ax.set_xlabel('Cache-Type')
    ax.set_title(f'{request_type} - CDN Cache Count')
    ax.legend(title='Cache-Type color')

    # Show the plot
    plt.show()

# ***************************************************************
# Note - FEEL FREE TO UPDATE BASED OO YOUR ANALYSIS REQUIREMENTS
# ***************************************************************

# Total Requests Analysis
total_count = len(df)
miss_count = (df['cache'] == 'MISS').sum()
hit_count =  (df['cache'] == 'HIT').sum()
pass_count =  (df['cache'] == 'PASS').sum()

if total_count > 0:
    analyze(total_count, miss_count, hit_count, pass_count, 'Total Requests')


# HTML Requests Analysis
total_count = (df['res_ctype'].str.startswith('text/html;')).sum()
miss_count = (df['res_ctype'].str.startswith('text/html;') & (df['cache'] == 'MISS')).sum() 
hit_count =  (df['res_ctype'].str.startswith('text/html;') & (df['cache'] == 'HIT')).sum() 
pass_count =  (df['res_ctype'].str.startswith('text/html;') & (df['cache'] == 'PASS')).sum() 

if total_count > 0:
    analyze(total_count, miss_count, hit_count, pass_count, 'HTML Requests')


# JS & CSS Requests Analysis
total_count = (df['res_ctype'].str.startswith('application/javascript;') | df['res_ctype'].str.startswith('text/css;')).sum()
miss_count = ((df['res_ctype'].str.startswith('application/javascript;') | df['res_ctype'].str.startswith('text/css;')) & (df['cache'] == 'MISS')).sum() 
hit_count =  ((df['res_ctype'].str.startswith('application/javascript;') | df['res_ctype'].str.startswith('text/css;')) & (df['cache'] == 'HIT')).sum() 
pass_count =  ((df['res_ctype'].str.startswith('application/javascript;') | df['res_ctype'].str.startswith('text/css;')) & (df['cache'] == 'PASS')).sum() 

if total_count > 0:
    analyze(total_count, miss_count, hit_count, pass_count, 'JavaScript & CSS Requests')


# Image Requests Analysis
total_count = (df['res_ctype'].str.startswith('image/')).sum()
miss_count = (df['res_ctype'].str.startswith('image/') & (df['cache'] == 'MISS')).sum() 
hit_count =  (df['res_ctype'].str.startswith('image/') & (df['cache'] == 'HIT')).sum() 
pass_count =  (df['res_ctype'].str.startswith('image/') & (df['cache'] == 'PASS')).sum() 

if total_count > 0:
    analyze(total_count, miss_count, hit_count, pass_count, 'Image Requests')


# JSON Requests Analysis
total_count = (df['res_ctype'].str.startswith('application/json')).sum()
miss_count = (df['res_ctype'].str.startswith('application/json') & (df['cache'] == 'MISS')).sum() 
hit_count =  (df['res_ctype'].str.startswith('application/json') & (df['cache'] == 'HIT')).sum() 
pass_count =  (df['res_ctype'].str.startswith('application/json') & (df['cache'] == 'PASS')).sum() 

if total_count > 0:
    analyze(total_count, miss_count, hit_count, pass_count, 'JSON Requests')


### Top 5 MISS and PASS Request URLs for HTML, JS/CSS and Image 

>
> **Note::**
> This information is helpful when your Cache Hit ratio is less than 90%. 
>


For each request type (HTML, JS/CSS, Image) show the top 5 MISS, and PASS request URLs and their counts. 


In [ ]:
from tabulate import tabulate

def displayTopURLsAndCounts(cache_type, request_type, no_of_results):

    # Filter rows where 'cache' is equal to and 'res_ctype' starts with
    filter1_df = df[df['cache'] == cache_type ]
    
    filter2_df = filter1_df[filter1_df['res_ctype'].str.startswith(request_type)]
    
    # Group by 'url' and count occurrences of 'cache' equal to 'PASS'
    result = filter2_df.groupby('url')['cache'].count().reset_index()
    
    # Sort the result in descending order by 'cache' count
    result = result.sort_values(by='cache', ascending=False)
    
    # Rename the columns
    result.columns = ['url', 'total_count']
    
    # Sort the result in descending order by 'total_count'
    result = result.sort_values(by='total_count', ascending=False)

    # Get the top results
    top_results = result.head(no_of_results)
    
    if top_results is not None:
        # Print the top results as a table with custom headings
        table = tabulate(top_results, headers='keys', tablefmt='grid', showindex="never")
        print(table)
    else:
        print('No data found with given criteria')


no_of_results = 5


print("\n\n")
print(colored("=================================================================================", "red", attrs=["bold"]))
print(colored(f" Top {no_of_results} HTML Requests of ", "red", attrs=["bold"]))
print(colored("=================================================================================", "red", attrs=["bold"]))

cache_type = 'MISS'
print(colored(f"\n{cache_type} Cache Type \n", "red", attrs=["bold"]))
displayTopURLsAndCounts(cache_type,'text/html',no_of_results)

cache_type = 'PASS'
print(colored(f"\n {cache_type} Cache Type \n", "red", attrs=["bold"]))
displayTopURLsAndCounts(cache_type,'text/html',no_of_results)


print("\n\n")
print(colored("=================================================================================", "red", attrs=["bold"]))
print(colored(f" Top {no_of_results} Image Requests of ", "red", attrs=["bold"]))
print(colored("=================================================================================", "red", attrs=["bold"]))

cache_type = 'MISS'
print(colored(f"\n{cache_type} Cache Type \n", "red", attrs=["bold"]))
displayTopURLsAndCounts(cache_type,'image/',no_of_results)

cache_type = 'PASS'
print(colored(f"\n {cache_type} Cache Type \n", "red", attrs=["bold"]))
displayTopURLsAndCounts(cache_type,'image/',no_of_results)


print("\n\n")
print(colored("=================================================================================", "red", attrs=["bold"]))
print(colored(f" Top {no_of_results} JavaScript Requests of ", "red", attrs=["bold"]))
print(colored("=================================================================================", "red", attrs=["bold"]))

cache_type = 'MISS'
print(colored(f"\n{cache_type} Cache Type \n", "red", attrs=["bold"]))
displayTopURLsAndCounts(cache_type,'application/javascript',no_of_results)

cache_type = 'PASS'
print(colored(f"\n {cache_type} Cache Type \n", "red", attrs=["bold"]))
displayTopURLsAndCounts(cache_type,'application/javascript',no_of_results)


print("\n\n")
print(colored("=================================================================================", "red", attrs=["bold"]))
print(colored(f" Top {no_of_results} CSS Requests of ", "red", attrs=["bold"]))
print(colored("=================================================================================", "red", attrs=["bold"]))

cache_type = 'MISS'
print(colored(f"\n{cache_type} Cache Type \n", "red", attrs=["bold"]))
displayTopURLsAndCounts(cache_type,'text/css',no_of_results)

cache_type = 'PASS'
print(colored(f"\n {cache_type} Cache Type \n", "red", attrs=["bold"]))
displayTopURLsAndCounts(cache_type,'text/css',no_of_results)


print("\n\n")
print(colored("=================================================================================", "red", attrs=["bold"]))
print(colored(f" Top {no_of_results} JSON Requests of ", "red", attrs=["bold"]))
print(colored("=================================================================================", "red", attrs=["bold"]))

cache_type = 'MISS'
print(colored(f"\n{cache_type} Cache Type \n", "red", attrs=["bold"]))
displayTopURLsAndCounts(cache_type,'application/json',no_of_results)

cache_type = 'PASS'
print(colored(f"\n {cache_type} Cache Type \n", "red", attrs=["bold"]))
displayTopURLsAndCounts(cache_type,'application/json',no_of_results)
